In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%mkdir -p git work

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd "/content/gdrive/My Drive/"
%cd "/content/gdrive/My Drive/git"
!git config --global user.email "louis.chevallier@gmail.com"
!git config --global user.name "louis-chevallier"

/content/gdrive/My Drive
/content/gdrive/My Drive/git


In [5]:
%pip install utillc
%load_ext autoreload
%autoreload 2
import utillc
from utillc import *
print(utillc.__version__)
print_everything()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 754 kB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 251 kB 11.3 MB/s 


without qt5
no qt5
0.9.20


In [4]:
!git clone https://github.com/ILikePython256/PyStereogram.git
%pwd

Cloning into 'PyStereogram'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 52 (delta 20), reused 33 (delta 9), pack-reused 0
Unpacking objects: 100% (52/52), done.


'/content/gdrive/MyDrive/git'

In [6]:
%cd PyStereogram/

/content/gdrive/MyDrive/git/PyStereogram


In [8]:
#Imports
from PIL import Image
import numpy as np
import sys

TILESIZE=50

#Get input
depthImName = sys.argv[1:2]
if depthImName == []:
    depthImName = input("Depthmap? ")
    depthImName = ("Depth.png" if depthImName == "" else depthImName)
else:
    depthImName = depthImName[0]

depthImName = 'Balls.png'

#Open depthmap
try:
    depth = np.array(Image.open(depthImName))
except FileNotFoundError:
    depth = np.array(Image.open("Input/" + depthImName))


In [9]:
print("Creating tile...")
import MakeBG
MakeBG.main(TILESIZE,depth.shape[1::-1])

print("Opening images...")
base = np.array(Image.open("BaseAuto.png"))
assert base.shape[:2] == depth.shape[:2], "Base and depthmap differ in size."

print("Creating arrays...")
depth -= depth.min()
depth = depth * ((TILESIZE / 3) / depth.max())
#depth = (depth * -1) + (depth.max())
diffmap = []
newImage = []
for i in range(base.shape[1]):
    diffmap.append([0] * base.shape[0])
    newImage.append([0] * base.shape[0])

print("Calculating shift...")
for x in range(len(diffmap)-1,-1,-1):
    for y in range(len(diffmap[0])):
        tmp = depth[y][x][0]
        if x < len(diffmap) - ((TILESIZE) + int(tmp)):
            tmp += diffmap[x+(TILESIZE)-int(tmp)][y]
        diffmap[x][y] = int(tmp)

print("Creating images...")
for x in range(len(diffmap)-1,-1,-1):
    for y in range(len(diffmap[0])):
        newImage[x][y] = tuple(base[y][(x-diffmap[x][y]) % base.shape[1]])

def bound(n):
    return int(
        (
            ((n + (TILESIZE/2)) % TILESIZE) - (TILESIZE/2)
        ) * 255/TILESIZE
    )


Creating tile...
Opening images...
Creating arrays...
Calculating shift...
Creating images...


In [11]:


difflist = []
for i in range(len(diffmap[0])):
    difflist += [(-bound(diffmap[j][i]), bound(diffmap[j][i]), 0) for j in range(len(diffmap))]

img = Image.new('RGB', (len(diffmap), len(diffmap[0])))
img.putdata(difflist)
img.save('trueDepth.png')

del difflist
im = []
for i in range(len(diffmap[0])):
    im += [newImage[j][i] for j in range(len(diffmap))]

img = Image.new('RGB', (len(diffmap), len(diffmap[0])))
img.putdata(im)
img.save('Output/%s' % depthImName.split("/")[-1])

print("Done!")

Done!
